<h3>🌟 Cleaned File names and Inner Connections Names</h3>

In [2]:
import os
import pandas as pd
import re
import unicodedata
import openpyxl

# Setup the folders where raw and cleaned data will be stored
raw_data_folder = "LinkedIn Data Public"  # Folder where raw data is stored
cleaned_data_folder = "Cleaned Data"  # Folder where cleaned data will be saved
os.makedirs(cleaned_data_folder, exist_ok=True)  # Create cleaned data folder if it doesn't exist

# Function to clean student name from the filename
def clean_student_name(filename):
    name_part = os.path.splitext(filename)[0]  # Remove file extension (.csv, .xlsx)

    parts = name_part.split(" - ")  # Split by " - " to get parts of the name

    # Check if filename starts with certain keywords (like 'LinkedIn' or 'Connections') and clean it
    if re.match(r'(?i)^(cleaned|Linked In|Linked_in_connection|Linkedin List|Linkedin_Connections|linkedin\s*connections?|connections?)', parts[0].strip()):
        candidate = parts[-1]
    else:
        candidate = parts[0]

    # Clean up: Replace underscores (_) and dashes (-) with spaces, remove unwanted words and digits
    candidate = candidate.replace("_", " ").replace("-", " ")
    candidate = re.sub(r'(?i)\b(cleaned|linkedin\s*connections?|connections?)\b', '', candidate)
    candidate = re.sub(r'\d+', '', candidate)  # Remove numbers
    candidate = re.sub(r'\.csv$', '', candidate, flags=re.IGNORECASE)  # Remove .csv extension
    candidate = " ".join(candidate.split())  # Remove extra spaces

    # If the candidate is empty or just the word "csv", use the original filename
    if not candidate or candidate.strip().lower() == 'csv':
        candidate = name_part

    return candidate.title()  # Return the cleaned name in title case

# Function to detect the header row in the dataset
def detect_header_row(df):
    for i, row in df.iterrows():
        lowered = row.astype(str).str.lower().tolist()
        # Check if columns contain "first name" and "last name" (case insensitive)
        if any("first" in col and "name" in col for col in lowered) and \
           any("last" in col and "name" in col for col in lowered):
            return i  # Return index of the header row
    return None  # Return None if no header row is found

# Function to remove special characters from names (such as accents and unwanted symbols)
def remove_special_characters(text):
    text = str(text).strip()
    
    # Normalize the text to remove accents and convert unicode characters
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    
    # Remove digits and symbols, but keep letters and spaces
    text = re.sub(r'[^A-Za-z\s]', '', text)
    
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    
    return text.title()  # Return the cleaned text in title case

# Function to load and clean the dataset (supports both CSV and Excel files)
def load_and_clean_file(file_path):
    ext = os.path.splitext(file_path)[1].lower()  # Get file extension (e.g., .csv or .xlsx)
    df_raw = None

    # If the file is CSV, try reading it with different encodings
    if ext == '.csv':
        for enc in ['utf-8', 'ISO-8859-1', 'utf-16']:  # Try multiple encodings
            try:
                df_try = pd.read_csv(file_path, header=None, encoding=enc)  # Read the file without header
                header_index = detect_header_row(df_try)  # Detect header row
                if header_index is not None:
                    df_raw = pd.read_csv(file_path, skiprows=header_index, encoding=enc)  # Read file with correct header
                    break
            except Exception:
                continue  # Skip if any error occurs during reading
    else:
        # If the file is Excel, read it similarly without a header
        try:
            df_try = pd.read_excel(file_path, header=None)
            header_index = detect_header_row(df_try)
            if header_index is not None:
                df_raw = pd.read_excel(file_path, skiprows=header_index)
        except Exception as e:
            print(f"❌ Error reading {os.path.basename(file_path)}: {e}")
            return None

    if df_raw is None:
        return None  # Return None if the file couldn't be loaded or cleaned

    # Clean column names by stripping extra spaces and converting to title case
    df_raw.columns = [str(c).strip().title() for c in df_raw.columns]
    # Check if the required columns 'First Name' and 'Last Name' exist
    if "First Name" not in df_raw.columns or "Last Name" not in df_raw.columns:
        print(f"⚠️ Skipping: 'First Name' or 'Last Name' column not found in {os.path.basename(file_path)}")
        return None

    # Extract 'First Name' and 'Last Name' columns and drop rows where both are empty
    df = df_raw[["First Name", "Last Name"]].dropna(how='all')

    # Clean up the names using the 'remove_special_characters' function
    df["First Name"] = df["First Name"].apply(remove_special_characters)
    df["Last Name"] = df["Last Name"].apply(remove_special_characters)
    # Create a 'Full Name' column by combining 'First Name' and 'Last Name'
    df["Full Name"] = df["First Name"] + " " + df["Last Name"]

    # Remove any duplicate full names
    df = df.drop_duplicates(subset=["Full Name"])
    return df  # Return the cleaned DataFrame

# Process all files in the raw data folder
seen_students = {}  # To keep track of already processed student names
lst = []  # To store the cleaned filenames

# Loop through all files in the raw data folder
for fname in sorted(os.listdir(raw_data_folder)):
    if not fname.lower().endswith(('.csv', '.xlsx', '.xls')):  # Skip non-CSV/Excel files
        continue

    file_path = os.path.join(raw_data_folder, fname)  # Get full file path
    student_name = clean_student_name(fname)  # Clean the student name from the filename

    # Skip if we've already processed this student
    if student_name in seen_students:
        continue
    seen_students[student_name] = fname  # Mark this student as processed

    df = load_and_clean_file(file_path)  # Load and clean the file
    if df is None or df.empty:  # Skip empty or invalid dataframes
        continue

    # Create a new filename for the cleaned data and save it
    cleaned_filename = f"{student_name}.csv"
    save_path = os.path.join(cleaned_data_folder, cleaned_filename)
    df[["Full Name"]].to_csv(save_path, index=False)  # Save only the 'Full Name' column
    lst.append(cleaned_filename[:-4])  # Add the cleaned filename (without .csv) to the list

# Function to rename files according to a given list of old and new filenames
def rename_files(folder, rename_list):
    for pair in rename_list:
        old_file = os.path.join(folder, pair[0])  # Old file path
        new_file = os.path.join(folder, pair[1])  # New file path

        # Check if the old file exists
        if os.path.isfile(old_file):
            try:
                os.rename(old_file, new_file)  # Rename the file
                print(f"{pair[0]} renamed to {pair[1]}")
            except Exception as err:
                print(f"Failed to rename {pair[0]}: {err}")
        else:
            print(f"{pair[0]} not found in {folder}")

# Folder where cleaned files are stored
data_dir = "Cleaned Data"

# List of files to rename (old name, new name)
files_to_rename = [
    ("Nirmal Linkdin.csv", "Nirmal Mewada.csv"),
    ("Byagaripraveen Kumar.csv", "Byagari Praveen Kumar.csv"),
    ("Himanshukanwarchundawat.csv", "Himanshu Kanwar Chundawat.csv"),
    ("Yuvrajsingh Bhati.csv", "Yuvraj Singh Bhati.csv"),
    ("Manoj K..csv", "Manoj Kharkar.csv"),
    ("N. Arun Kumar.csv", "Nallamothu Arun Kumar.csv")
]

# Call the function to rename the files
rename_files(data_dir, files_to_rename)

# Print the number of cleaned files processed
print(len(lst))  # Print how many files were processed and cleaned


Nirmal Linkdin.csv renamed to Nirmal Mewada.csv
Byagaripraveen Kumar.csv renamed to Byagari Praveen Kumar.csv
Himanshukanwarchundawat.csv renamed to Himanshu Kanwar Chundawat.csv
Yuvrajsingh Bhati.csv renamed to Yuvraj Singh Bhati.csv
Manoj K..csv renamed to Manoj Kharkar.csv
N. Arun Kumar.csv renamed to Nallamothu Arun Kumar.csv
126


<h3>🌟Number of cleaned files in folder "Cleaned Data"</h3>

In [3]:
import os

# Define the path to the folder where the files are stored
folder_path = "Cleaned Data"  # Replace this with the actual folder path where your files are stored

# Count the number of files in the folder
file_count = sum(1 for fname in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, fname)))

# Print the total number of files in the folder
print(f"Total files in {folder_path}: {file_count}")


Total files in Cleaned Data: 126


<h3>🌟All Names Of Files saved in a Text File</h3>

In [4]:
import os

# Function to save all filenames in the specified folder to a text file
def save_filenames_to_text(folder):
    # Define the output file path (it will be saved as .all_filenames.txt in the folder)
    output_file = os.path.join(folder, ".all_filenames.txt")
    
    try:
        # Open the output file in write mode with UTF-8 encoding
        with open(output_file, 'w', encoding='utf-8') as file:
            # Loop through each item in the folder
            for filename in os.listdir(folder):
                # Check if the item is a file (not a directory) and not the .all_filenames.txt file
                if os.path.isfile(os.path.join(folder, filename)) and filename != ".all_filenames.txt":
                    # Write the filename to the text file, followed by a newline
                    file.write(filename + '\n')
        
        # Print a success message with the output file path
        print(f"All file names saved to: {output_file}")
    
    except Exception as e:
        # If there's an error, print the error message
        print(f"Error writing file names: {e}")

# Folder path where the files are located
folder_path = "Cleaned Data"
# Call the function to save the filenames to the text file
save_filenames_to_text(folder_path)


All file names saved to: Cleaned Data\.all_filenames.txt
